In [1]:
!git clone https://github.com/openai/gpt-3.git

Cloning into 'gpt-3'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 46 (delta 4), reused 0 (delta 0), pack-reused 38
Unpacking objects: 100% (46/46), done.


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.6 MB/s 
     |████████████████████████████████| 596 kB 52.0 MB/s 
     |████████████████████████████████| 101 kB 12.6 MB/s 
     |████████████████████████████████| 6.6 MB 33.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 8.9 MB/s 


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 

tokenizer = AutoTokenizer.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)
model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)

_ = model.eval()


#prompt: 입력, 최대 2048 tokens (1500 단어)
#completion: 생성된 출력, 최대 2048 tokens (1500 단어)
#tokens: 말뭉치 조각 - 단어와 비슷함, 많은 토큰이 공백으로 시작함 “ hello”
#prompt 끝부분에 공백을 놔두면 안됨.
prompt = '''
동요 노래 가사

1.파란 하늘 파란 하늘 꿈이 드리운 푸른 언덕에 아기 염소 여럿이 풀을 뜯고 놀아요 해처럼 밝은 얼굴로

2.학교 종이 땡땡땡 어서모이자 선생님이 우리를 기다리신다

3.산토끼 토끼야 어디를 가느냐 깡총깡총 뛰면서 어디를 가느냐

4.떴다 떴다 비행기 날아라 날아라 높이 높이 날아라 우리 비행기

5.새신을 신고 뛰어보자 팔짝 머리가 하늘까지 닿겠네

6.곰 세 마리가 한 집에 있어 아빠곰 엄마곰 애기곰 아빠곰은 뚱뚱해 엄마곰은 날씬해 애기곰은 너무 귀여워 으쓱으쓱 잘한다

7.'''

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=350)
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)



동요 노래 가사

1.파란 하늘 파란 하늘 꿈이 드리운 푸른 언덕에 아기 염소 여럿이 풀을 뜯고 놀아요 해처럼 밝은 얼굴로

2.학교 종이 땡땡땡 어서모이자 선생님이 우리를 기다리신다

3.산토끼 토끼야 어디를 가느냐 깡총깡총 뛰면서 어디를 가느냐

4.떴다 떴다 비행기 날아라 날아라 높이 높이 날아라 우리 비행기

5.새신을 신고 뛰어보자 팔짝 머리가 하늘까지 닿겠네

6.곰 세 마리가 한 집에 있어 아빠곰 엄마곰 애기곰 아빠곰은 뚱뚱해 엄마곰은 날씬해 애기곰은 너무 귀여워 으쓱으쓱 잘한다

7.구름을 잡아라 냇물을 따라 가면 내가 놀던 물이 흘러 어디로 가는지 알 수 있지

8.햇님이 방긋 웃으면 나는야 햇님이 얼굴

9.비 오는 날 우산 속 얼굴 마주하고 너랑 나랑 둘이 비를 맞네

10.새나라의 어린이는 일찍 일어납니다 학교 가는 길에 달그락 달그락

11.눈 오는 날 눈 싸움을 해요 동무들과 모여서 눈 싸움을 해요 눈 싸움을 해요

12.누나 손을 잡고서 노래를 불러요 동동동동 손을 잡고서

13.산에 올라요 산에 올라요 푸른 하늘 푸른 한울 푸른 언덕에

14.이 다음에 커서 어른 되면 나도 어른 되면

15.저 건너 푸른 언덕에 아기 염소 여럿이 풀을 뜯고 놀아요 해처럼 밝은 얼굴로


